In [29]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import texttable
import latextable
import json

tags = {
    "loss": "Loss_MB/train_phase/train_stream/Task000",
    "final_accuracy": "Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000",
}

def load_event_file(experiment_directory: str) -> EventAccumulator:
    """
    Loads the event file from a given experiment directory.
    """
    ea: EventAccumulator = EventAccumulator(experiment_directory)
    ea.Reload()
    return ea

def final_metrics(ea: EventAccumulator) -> dict:
    """
    Returns the final metrics from the event file.
    """
    result = {}
    for tag_name, tag in tags.items():
        if tag in ea.Tags()['scalars']:
            result[tag_name] = ea.Scalars(tag)[-1].value
    return result



In [31]:
experiment_logs = "experiment_logs"

records = []

# Loop over each directory in a directory
for experiment in tqdm(os.listdir(os.path.join(experiment_logs))):
    experiment_code = experiment.split("_")
    i, repo_hash, experiment_category, dataset, arch, strategy = experiment_code


    record = {
        "dataset": dataset,
        "architecture": arch,
        "strategy": strategy,
        "experiment_category": experiment_category,
        "id": i,
        "repo_hash": repo_hash
    }

    try:
        ea = load_event_file(os.path.join(experiment_logs, experiment))
        record.update(final_metrics(ea))
    except KeyError as e:
        print(f"Could not load required metric {e} in '{experiment}'")

    try:
        with open(os.path.join(experiment_logs, experiment, "config.json"), "r") as f:
            config = json.load(f)
            record.update(config)
    except FileNotFoundError as e:
        print(f"Could not load config file '{experiment}/config.json'")

    records.append(record)

df = pd.DataFrame.from_dict(records)
df

  0%|          | 0/149 [00:00<?, ?it/s]

Could not load config file [Errno 2] No such file or directory: 'experiment_logs/0004_dd88ddf_N_splitFMNIST_VAE_taskOracle/config.json' in '0004_dd88ddf_N_splitFMNIST_VAE_taskOracle'
Could not load config file [Errno 2] No such file or directory: 'experiment_logs/0006_dd88ddf_N_splitFMNIST_VAE_taskInference/config.json' in '0006_dd88ddf_N_splitFMNIST_VAE_taskInference'
Could not load config file [Errno 2] No such file or directory: 'experiment_logs/0002_dd88ddf_N_splitFMNIST_VAE_cumulative/config.json' in '0002_dd88ddf_N_splitFMNIST_VAE_cumulative'
Could not load config file [Errno 2] No such file or directory: 'experiment_logs/0101_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskOracle/config.json' in '0101_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskOracle'
Could not load config file [Errno 2] No such file or directory: 'experiment_logs/0082_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json' in '0082_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference'
Could not load config file [Errno 2] 

,dataset,architecture,strategy,prune_strategy,id,repo_hash,loss,final_accuracy,batch_size,classifier_loss_weight,...,total_task_epochs,use_classifier_loss,use_packnet,use_reconstruction_loss,use_vae_loss,vae_loss_weight,vanilla_cnn_config,embedding_module,recon_loss_type,rectangular_mlp_config
0,splitCIFAR10,AE,taskInference,N,0040,dd88ddf,0.342900,0.712300,64.0,1.0,...,50.0,True,True,True,False,0.001,{'base_channels': 128},NaN,NaN,NaN
1,splitEmbeddedCIFAR100,AE,taskInference,N,0099,dd88ddf,1.020918,0.685600,64.0,1.0,...,100.0,True,True,True,False,0.001,NaN,ResNet18,mse,NaN
2,splitCIFAR10,AE,taskInference,PL,0158,55dff8e1,NaN,0.593900,64.0,1.0,...,50.0,True,True,True,False,NaN,NaN,None,bce,NaN
3,splitEmbeddedCORe50,AE,taskOracle,N,0116,dd88ddf,NaN,0.604243,64.0,1.0,...,10.0,True,True,True,False,0.001,NaN,ResNet50,mse,NaN
4,splitEmbeddedCIFAR100,VAE,cumulative,N,0109,dd88ddf,NaN,0.397400,64.0,1.0,...,100.0,True,False,True,True,0.001,NaN,ResNet50,mse,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,splitEmbeddedCIFAR100,VAE,taskInference,N,0121,dd88ddf,NaN,0.362600,64.0,1.0,...,100.0,True,True,True,True,0.001,NaN,ResNet50,mse,NaN
145,splitFMNIST,AE,finetuning,N,0023,dd88ddf,0.151736,0.199500,64.0,1.0,...,20.0,True,False,True,False,NaN,{'base_channels': 128},NaN,NaN,NaN
146,splitCIFAR10,VAE,finetuning,N,0028,dd88ddf,0.196268,0.194500,64.0,1.0,...,50.0,True,False,True,True,0.001,{'base_channels': 128},NaN,NaN,NaN
147,splitFMNIST,AE,cumulative,N,0014,dd88ddf,0.157962,0.915100,64.0,1.0,...,20.0,True,False,True,False,NaN,{'base_channels': 128},NaN,NaN,NaN


In [3]:
df.dropna().sort_values(by='id').groupby(['dataset', 'architecture', 'strategy']).max().pivot_table(index=['architecture', 'strategy'], columns=['dataset'], values='final_accuracy').to_csv('accuracy.csv')

In [4]:
df.to_csv("results/results.csv")

In [32]:
# 
# Regular Experiment
# 
for strategy in ["cumulative", "taskOracle", "finetuning", "taskInference"]:
    for arch in ["AE", "VAE"]:
        for dataset in [
            "splitFMNIST",
            "splitCIFAR10",
            "splitCIFAR100",
            "splitCORe50",
            "splitEmbeddedCIFAR100",
            "splitEmbeddedCORe50"]:
            try:
                result = df[df['dataset'] == dataset]
                result = result[result['prune_strategy'] == "EXP"]
                result = result[result['strategy'] == strategy]
                result = result[result['architecture'] == arch]
                accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                # accuracy = result.sort_values("id")[-1]["accuracy"]
                # display(result.sort_values(by='id'))
                print(f"{accuracy:0.4f}", end="\t")
            except:
                continue
        print()

0.9151	0.8721	0.6167	0.3560	0.4210	0.7720	
0.8989	0.8725	0.6106	0.3604	0.3974	0.7593	
0.9938	0.9375	0.5475	0.4322	0.5513	0.6042	
0.9936	0.9382	0.5910	0.4032	0.4735	0.5191	
0.1995	0.1939	0.0899	0.0745	0.0841	0.0881	
0.1995	0.1945	0.0891	0.0751	0.0849	0.0578	
0.9597	0.7123	0.2351	0.0553	0.4392	0.4191	
0.9927	0.7295	0.3091	0.0531	0.3626	0.3756	


In [39]:
# 
# Equal Prune Experiment
# 
def table_equal_prune(experiment_results: pd.DataFrame):
    df = experiment_results.copy()

    equal_prune_table = texttable.Texttable()
    columns = ["splitFMNIST", "splitCIFAR10", "splitCIFAR100", "splitCORe50", "splitEmbeddedCIFAR100", "splitEmbeddedCORe50"]
    
    equal_prune_table.header(["", ""] + columns)

    for strategy in ["taskOracle", "taskInference"]:
        for arch in ["AE", "VAE"]:
            row = [strategy, arch]
            for dataset in columns:
                try:
                    result = df.loc[
                        (df['dataset'] == dataset) &
                        (df['prune_strategy'] == "EP") &
                        (df['strategy'] == strategy) &
                        (df['architecture'] == arch)
                    ]
                    accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                    row.append(f"{accuracy*100:0.1f}\%")
                except:
                    row.append(f"NA")
            equal_prune_table.add_row(row)

    print(equal_prune_table.draw())

    print(latextable.draw_latex(
        equal_prune_table,
        caption="Equal Prune Experiment",
        label="tab:equal_prune_experiment",
        use_booktabs=True
    ))

table_equal_prune(df[df["prune_strategy"]=="EP"])

+----------+-----+----------+----------+---------+---------+---------+---------+
|          |     | splitFMN | splitCIF | splitCI | splitCO | splitEm | splitEm |
|          |     |   IST    |   AR10   | FAR100  |  Re50   | beddedC | beddedC |
|          |     |          |          |         |         | IFAR100 |  ORe50  |
+==========+=====+==========+==========+=========+=========+=========+=========+
| taskOrac | AE  | 99.3\%   | 93.4\%   | 55.7\%  | 50.7\%  | NA      | NA      |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskOrac | VAE | 99.3\%   | 91.7\%   | 53.0\%  | 54.1\%  | NA      | NA      |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskInfe | AE  | 99.3\%   | 43.0\%   | 25.7\%  | 8.2\%   | NA      | NA      |
| rence    |     |          

In [47]:
# 
# Equal Prune Experiment
# 
def table_prune_levels(experiment_results: pd.DataFrame):
    df = experiment_results.copy()

    equal_prune_table = texttable.Texttable()
    columns = ["splitFMNIST", "splitCIFAR10", "splitCIFAR100", "splitCORe50", "splitEmbeddedCIFAR100", "splitEmbeddedCORe50"]
    
    equal_prune_table.header(["Prune Level"] + columns)

    for prune_level in [0.2, 0.4, 0.5, 0.6, 0.8]:
            row = [f"{prune_level*100:0.0f}\%"]
            for dataset in columns:
                try:
                    result = df.loc[
                        (df['dataset'] == dataset) &
                        (df['prune_proportion'] == prune_level) &
                        (df['architecture'] == "AE")
                    ]
                    accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                    row.append(f"{accuracy*100:0.1f}\%")
                except:
                    row.append(f"NA")
            equal_prune_table.add_row(row)

    print(equal_prune_table.draw())

    print(latextable.draw_latex(
        equal_prune_table,
        caption="Prune Levels Task Inference AutoEncoder",
        label="tab:prune_levels",
        use_booktabs=True
    ))

table_prune_levels(df[(df["prune_strategy"]=="PL") & (df["repo_hash"]=="55dff8e1")])


+-----------+-----------+----------+----------+----------+----------+----------+
|   Prune   | splitFMNI | splitCIF | splitCIF | splitCOR | splitEmb | splitEmb |
|   Level   |    ST     |   AR10   |  AR100   |   e50    | eddedCIF | eddedCOR |
|           |           |          |          |          |  AR100   |   e50    |
+===========+===========+==========+==========+==========+==========+==========+
| 20\%      | 73.6\%    | 36.7\%   | 17.1\%   | 6.7\%    | NA       | NA       |
+-----------+-----------+----------+----------+----------+----------+----------+
| 40\%      | 99.4\%    | 43.5\%   | 21.5\%   | 5.9\%    | NA       | NA       |
+-----------+-----------+----------+----------+----------+----------+----------+
| 50\%      | 99.3\%    | 56.3\%   | 28.7\%   | 6.4\%    | NA       | NA       |
+-----------+-----------+----------+----------+----------+----------+----------+
| 60\%      | 99.4\%    | 78.6\%   | 17.3\%   | 11.7\%   | NA       | NA       |
+-----------+-----------+---